In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pathlib
from sklearn.model_selection import train_test_split

In [2]:
# Connecting to Drive
import google
google.colab.drive.mount('/content/drive', force_remount=True)
Main_dir="drive/MyDrive/NASA_Turbofan/"

Mounted at /content/drive


In [3]:
# Function to make Train
def make_df_train(path_to_sensor_file,set_num):
  columns=["Engine_ID","Cycle","op1","op2","op3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","Other1","Other2"]
  

  df_sensors=pd.read_csv(path_to_sensor_file,sep=" ",names=columns)
  df_sensors.drop(['Other1','Other2'], axis=1,inplace=True)

  df_last_cycle=df_sensors.drop_duplicates(subset=['Engine_ID'],keep='last')
  last_cycle=df_last_cycle['Cycle'].tolist()
 
  df_RUL= pd.DataFrame()
  df_RUL["RUL_starting"]=last_cycle
  Eng_ids=[i+1 for i in range(len(last_cycle))]
  df_RUL["Engine_ID"]=Eng_ids

  df_merged=pd.merge(df_sensors, df_RUL, on='Engine_ID')
  df_merged['RUL'] = df_merged['RUL_starting'] - df_merged['Cycle'] 
  df_merged.drop(['RUL_starting'], axis=1,inplace=True)
  engine_id_with_operation_datset=df_merged["Engine_ID"].tolist()
  new_ids_generated=[str(i)+"_"+str(set_num) for i in engine_id_with_operation_datset]
  df_merged["Engine_ID"]=new_ids_generated

  # Making Binary classification labels - less than 10 cycles - Critical |  >10 cycles - Working | Conditions : 0-Critical | 1-Working 
  df_merged['Status']=df_merged['RUL'].apply(lambda x: 0 if x <= 10 else 1)

  # Making status subclasses - | Conditions : Critical | Warning | Working 
  df_merged['Sub_Status']=df_merged['RUL'].apply(lambda x: 'Critical' if x <= 10 else ('Warning' if x<=30 else 'Working'))

  display(df_merged)

  return df_merged

In [4]:
# First Dataset
dataset_num=1
path_to_sensor_file=Main_dir+"train_FD001.txt"
DS_1=make_df_train(path_to_sensor_file,dataset_num)

,Engine_ID,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status,Sub_Status
0,1_1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,1,Working
1,1_1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,1,Working
2,1_1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,1,Working
3,1_1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,1,Working
4,1_1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,1,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100_1,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,8.4956,0.03,397,2388,100.0,38.49,22.9735,4,0,Critical
20627,100_1,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,8.5139,0.03,395,2388,100.0,38.30,23.1594,3,0,Critical
20628,100_1,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,8.5646,0.03,398,2388,100.0,38.44,22.9333,2,0,Critical
20629,100_1,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,8.5389,0.03,395,2388,100.0,38.29,23.0640,1,0,Critical


In [5]:
# Second dataset
dataset_num=2
path_to_sensor_file=Main_dir+"train_FD002.txt"
DS_2=make_df_train(path_to_sensor_file,dataset_num)

,Engine_ID,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status,Sub_Status
0,1_2,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,9.3461,0.02,334,2223,100.00,14.73,8.8071,148,1,Working
1,1_2,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,9.3774,0.02,330,2212,100.00,10.41,6.2665,147,1,Working
2,1_2,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,10.8941,0.02,309,1915,84.93,14.08,8.6723,146,1,Working
3,1_2,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,9.3528,0.02,329,2212,100.00,10.59,6.4701,145,1,Working
4,1_2,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,10.8963,0.02,309,1915,84.93,14.13,8.5286,144,1,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260_2,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,9.3035,0.03,369,2324,100.00,24.36,14.5189,4,0,Critical
53755,260_2,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,8.7586,0.03,374,2319,100.00,28.10,16.9454,3,0,Critical
53756,260_2,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,11.0657,0.02,310,1915,84.93,14.19,8.5503,2,0,Critical
53757,260_2,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,11.0537,0.02,311,1915,84.93,14.05,8.3729,1,0,Critical


In [6]:
# Third dataset
dataset_num=3
path_to_sensor_file=Main_dir+"train_FD003.txt"
DS_3=make_df_train(path_to_sensor_file,dataset_num)

,Engine_ID,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status,Sub_Status
0,1_3,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,8.4246,0.03,391,2388,100.0,39.11,23.3537,258,1,Working
1,1_3,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,8.4403,0.03,392,2388,100.0,38.99,23.4491,257,1,Working
2,1_3,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,8.3901,0.03,391,2388,100.0,38.85,23.3669,256,1,Working
3,1_3,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,8.3878,0.03,392,2388,100.0,38.96,23.2951,255,1,Working
4,1_3,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,8.3869,0.03,392,2388,100.0,39.14,23.4583,254,1,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100_3,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,8.5036,0.03,394,2388,100.0,38.44,22.9631,4,0,Critical
24716,100_3,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,8.5174,0.03,395,2388,100.0,38.50,22.9746,3,0,Critical
24717,100_3,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,8.5223,0.03,396,2388,100.0,38.39,23.0682,2,0,Critical
24718,100_3,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,8.5148,0.03,395,2388,100.0,38.31,23.0753,1,0,Critical


In [7]:
# Fourth dataset
dataset_num=4
path_to_sensor_file=Main_dir+"train_FD004.txt"
DS_4=make_df_train(path_to_sensor_file,dataset_num)

,Engine_ID,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status,Sub_Status
0,1_4,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,9.3335,0.02,330,2212,100.00,10.62,6.3670,320,1,Working
1,1_4,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,9.1913,0.02,361,2324,100.00,24.37,14.6552,319,1,Working
2,1_4,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,9.4007,0.02,329,2212,100.00,10.48,6.4213,318,1,Working
3,1_4,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,9.3369,0.02,328,2212,100.00,10.54,6.4176,317,1,Working
4,1_4,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,10.8366,0.02,305,1915,84.93,14.03,8.6754,316,1,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249_4,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,8.4541,0.03,372,2319,100.00,29.11,17.5234,4,0,Critical
61245,249_4,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,8.2221,0.03,396,2388,100.00,39.38,23.7151,3,0,Critical
61246,249_4,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,8.2525,0.03,395,2388,100.00,39.78,23.8270,2,0,Critical
61247,249_4,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,9.0515,0.02,337,2223,100.00,15.26,9.0774,1,0,Critical


In [8]:
# concatenting all four datasets
Final_df=pd.concat([DS_1,DS_2,DS_3,DS_4])
display(Final_df)

,Engine_ID,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status,Sub_Status
0,1_1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,1,Working
1,1_1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,1,Working
2,1_1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,1,Working
3,1_1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,1,Working
4,1_1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,1,Working
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249_4,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,8.4541,0.03,372,2319,100.0,29.11,17.5234,4,0,Critical
61245,249_4,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,8.2221,0.03,396,2388,100.0,39.38,23.7151,3,0,Critical
61246,249_4,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,8.2525,0.03,395,2388,100.0,39.78,23.8270,2,0,Critical
61247,249_4,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,9.0515,0.02,337,2223,100.0,15.26,9.0774,1,0,Critical


In [11]:
Final_df.describe()


,Cycle,op1,op2,op3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,RUL,Status
count,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,...,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000,160359.000000
mean,123.331338,17.211973,0.410004,95.724344,485.840890,597.361022,1467.035653,1260.956434,9.894999,14.424935,...,8088.950972,9.054747,0.025185,360.698801,2273.754039,98.389146,25.942709,15.565700,122.331338,0.951365
std,83.538146,16.527988,0.367938,12.359044,30.420388,42.478516,118.175261,136.300073,4.265554,6.443922,...,80.623257,0.751581,0.004997,31.021430,142.513114,4.656270,11.691422,7.015067,83.538146,0.215104
min,1.000000,-0.008700,-0.000600,60.000000,445.000000,535.480000,1242.670000,1023.770000,3.910000,5.670000,...,7845.780000,8.156300,0.020000,302.000000,1915.000000,84.930000,10.160000,6.010500,0.000000,0.000000
25%,57.000000,0.001300,0.000200,100.000000,449.440000,549.960000,1357.360000,1126.830000,5.480000,8.000000,...,8070.530000,8.439250,0.020000,332.000000,2212.000000,100.000000,14.330000,8.601300,56.000000,1.000000
50%,114.000000,19.998100,0.620000,100.000000,489.050000,605.930000,1492.810000,1271.740000,9.350000,13.660000,...,8118.590000,9.030100,0.030000,367.000000,2319.000000,100.000000,24.920000,14.953500,113.000000,1.000000
75%,173.000000,35.001500,0.840000,100.000000,518.670000,642.340000,1586.590000,1402.200000,14.620000,21.610000,...,8139.410000,9.344200,0.030000,392.000000,2388.000000,100.000000,38.820000,23.294600,172.000000,1.000000
max,543.000000,42.008000,0.842000,100.000000,518.670000,645.110000,1616.910000,1441.490000,14.620000,21.610000,...,8293.720000,11.066900,0.030000,400.000000,2388.000000,100.000000,39.890000,23.950500,542.000000,1.000000


In [ ]:
#Saving the dataframe to csv file
Final_df.to_csv(Main_dir+"Master_Data.csv")  

In [ ]:
# Doing train test split - 80% 20 % - and saving those files 
train, test = train_test_split(Final_df, test_size=0.2,random_state=3,shuffle=True,stratify=Final_df[["Engine_ID","Status", "Sub_Status"]])

In [ ]:
train.to_csv(Main_dir+"Master_train.csv")  
test.to_csv(Main_dir+"Master_test.csv")

In [ ]:
print(len(train))
print(len(test))

128287
32072
